<a href="https://colab.research.google.com/github/TollanBerhanu/Semantic-search-on-Slack/blob/main/Slack_Semantic_Search_(Instructor_Embeddings_%2B_Chroma_DB_%2B_LLaMA2_7B).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementing Semantic Search on Exported Slack Data

This notebook contains an implementation of semantic search on exported slack data

*   *This notebook utilizes the following tools:*

>

    1.   'Pandas' - to load and extract relevant information from the exported data
    2.   'Instructor embedding model' - to generate embeddings for each message
    3.   'Chroma' - to store and query the vector embeddings along with some metadata
    4.   'LLaMA2 7B model' - to present the results in natural language


## 1. Installing dependencies

In [1]:
# Install gitpython to clone a github repo containing the exported slack data
!pip install gitpython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [2]:
#Install dependencies for Llama 2
!pip -q install git+https://github.com/huggingface/transformers # need to install transformers from github
!pip install -q datasets loralib sentencepiece
!pip -q install bitsandbytes accelerate xformers einops

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 8.

In [3]:
# Install Instructor Embedding and Chroma
!pip -q install langchain chromadb sentence_transformers InstructorEmbedding

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.0/399.0 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 79.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.7 MB/s eta 0:00:00
  Installing build depen

In [4]:
# Install ngrok to host an api endpoint from colab
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19866 sha256=41bd7081aeb49031f05917778a78c3d7a6d6f79adfee41c86004f7aee6365d0a
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


## 2. Fetching the slack data

In [5]:
# The path where the exported slack data is stored in local storage
slack_data_path = "/content/slackdata/"

Run this if you haven't already cloned the GitHub repository

In [6]:
import git

repo_url = "https://github.com/TollanBerhanu/MatterMost-LLM-test-Slack-export-Jun-19-2023---Jun-20-2023.git"
# slack_data_path = '/content/drive/MyDrive/Colab Notebooks/dataset/slack-data/'

git.Repo.clone_from(repo_url, slack_data_path)

<git.repo.base.Repo '/content/slackdata/.git'>

In [7]:
import os
import pandas as pd

def get_all_channels(path):
  df = pd.read_json(path + 'channels.json')

  channel_ids = [id for id in df['id']]
  channel_names = [ name for name in df['name']]

  return pd.DataFrame({ 'channel_id': channel_ids, 'channel_name': channel_names } )

channels = get_all_channels(slack_data_path)
channels

,channel_id,channel_name
0,C05D1SE01B7,random
1,C05D77W3N76,general
2,C05D7863DRA,test
3,C05ABCDE01,gptgenerated


In [11]:
import glob
import json

# Return the metadata of each message in the channel
def extract_channel_metadata(path, channel_name):

  daily_json_files = glob.glob(path + channel_name +'/*.json')  # use glob to get all the json files in the folder

  if not daily_json_files:  # return if the channel doesn't exist (or hasn't been exported yet)
    return

  metadata = pd.DataFrame(columns = ['message', 'channel', 'date', 'time', 'user_id', 'user_name'])

  # loop over the list of json files (each json file includes every message in that channel for a single day)
  for f in daily_json_files:
    with open(f, 'r') as file:  # open the daily json file
        data = file.read()  # Read the contents
        today_data = json.loads(data) # Parse the JSON data

    today_date = f.split("/")[-1]  # 'f' is the full file path and file name
    print('Extracting...', today_date) # the file name is the date

    # iterate through all the messages of the day
    for msg_data in today_data:
      # Skip if its a "channel_join" type message or if the actual message content is empty
      if ('subtype' in msg_data) or (msg_data['text'] == "") or (msg_data['type'] != 'message'):
        continue
        # TODO: filter out any links, stickers, and other junk
        # TODO: replace @Member references with their real names

      metadata.loc[len(metadata)] = {
            'message': msg_data['user_profile']['first_name'] + ': ' + msg_data['text'],
            'channel': channel_name,
            'date': today_date.split(".json")[0], # omit the file extension '.json'
            'time': msg_data['ts'],
            'user_id': msg_data['user'],
            'user_name': msg_data['user_profile']['real_name'] # We can use 'first_name' to get the first name and 'real_name' to get the full name of the user
      }

  return metadata

# extract_channel_metadata(slack_data_path, 'random')

## 3. Generating Embeddings

In [13]:
from InstructorEmbedding import INSTRUCTOR
from langchain.embeddings import HuggingFaceInstructEmbeddings

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange


In [14]:
# Load the embedding model
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                                      model_kwargs={"device": "cuda"})

load INSTRUCTOR_Transformer
max_seq_length  512


In [30]:
# Get the list of embeddings for all messages in a channel
def embed_channel_messages(channel_messages):
  msg_list = channel_messages.astype(str).tolist()
  return instructor_embeddings.embed_documents(msg_list)

## 4. Storing the embeddings in Chroma DB


In [16]:
import chromadb

In [17]:
client = chromadb.PersistentClient(path="/content/chroma_db")

In [18]:
# Get a collection object from an existing collection, by name. If it doesn't exist, create one.
collection = client.get_or_create_collection(
      name= "slack_collection",
      metadata= {"hnsw:space": "cosine"},
      # embedding_function= instructor_embeddings       # The default embedding model is 'all-MiniLM-L6-V2'
    )

In [15]:
# Warning: Delete a collection and all associated embeddings, documents, and metadata. ⚠️ This is destructive and not reversible :(
# client.delete_collection(name="slack_collection")

In [27]:
def upsert_channel_embeddings(channel_embeddings, channel_metadata):

  # parse the channel metadata to json
  parsed_channel_metadata = json.loads(channel_metadata.to_json(orient="records"))

  #
  channel_name_list = channel_metadata['channel'].astype(str).tolist()

  # create IDs for the embeddings ... [channelname_0 -> ... -> channelname_..]
  ids = [ (channel_name_list[ch] + str(ch)) for ch in enumerate(channel_name_list) ]

  # upsert the embeddings along with their metadata, into a Chroma collection
  collection.upsert(
    ids = ids,
    embeddings = channel_embeddings,
    metadatas = parsed_channel_metadata,
    documents = channel_name_list
  )

  print(collection.peek()) # returns a list of the first 10 items in the collection
  print(collection.count()) # returns the number of items in the collection

In [28]:
# Upsert all channel's data into the vector db
def upsert_all_channels():
  channel_names = channels['channel_name'].tolist()
  for channel_name in channel_names:

    channel_metadata = extract_channel_metadata(slack_data_path, channel_name)

    channel_embeddings = embed_channel_messages(channel_metadata['message'])

    upsert_channel_embeddings(channel_embeddings, channel_metadata)

# Upsert just one channel's data
def upsert_one_channel(channel_name):

  channel_metadata = extract_channel_metadata(slack_data_path, channel_name)

  channel_embeddings = embed_channel_messages(channel_metadata)

  upsert_channel_embeddings(channel_embeddings, channel_metadata)

In [31]:
# upsert_all_channels()
upsert_one_channel('random')  # general, random, gptgenerated

Extracting... 2023-06-19.json


AttributeError: ignored

In [20]:
# Load the persisted database from disk, and use it as normal.
# vectordb = Chroma(persist_directory= '/chroma/db')

## 5. Querying the messages from Chroma

In [46]:
def get_data_from_chroma(query):
  # Generate embeddings for the query
  embedded_query = instructor_embeddings.embed_query(query)

  query_response = collection.query(
      query_embeddings = embedded_query,
      n_results = 5,
      # where = {"metadata_field": "is_equal_to_this"},
      where = {
          # "channel": {"$eq": "general"}
          # "user_id": {"$in": ["U05D1SQDNSH", "U05DHDPL4FK", "U05CQ93C3FZ", "U05D4M7RGQ3"]}
      }
      # where_document={"$contains":"search_string"}
  )

  # documents = query_response['documents']
  scores = query_response['distances'][0]
  metadatas = query_response['metadatas'][0]

  # print(scores, metadatas)

  context = ''
  metadatas = []

  for idx, metadata in query_response['metadatas'][0]:
    context += metadata['message'] + '\n'
    metadata['score'] = scores[]

  return {'context': context, 'score': scores, 'metadata': metadatas}

# get_context_from_chroma("Why was it good work?")
# get_context_from_chroma("Why did Tollan say it was good work?")
# get_context_from_chroma("What are some models that are comparable to GPT 3?")

## 6. Getting the response from LLaMA 2 - 7B

In [22]:
!nvidia-smi

Thu Jul 27 12:16:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    34W /  70W |   6005MiB / 15360MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [23]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

In [27]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                          use_auth_token=True,)

model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf",
                                             device_map='auto',
                                             torch_dtype=torch.float16,
                                             use_auth_token=True,
                                            #  load_in_8bit=True,
                                             load_in_4bit=True
                                             )

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:628: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:460: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:373: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [28]:
# Use a pipeline for later
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )

In [29]:
import json
import textwrap

'''
  You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful,
  unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

  If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question,
  please don't share false information. Don't provide any information you weren't asked to provide.
'''

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
  You will be given a sequence of chat messages related to a certain topic. Write a response that answers the question based on what is discussed in the chat messages.
  Do not mention anything outside of what is discussed. Don't answer anything outside the context you are provided.
  If there isn't enough context, simply reply "This topic was not discussed previously"
  """

SYSTEM_PROMPT = B_SYS + DEFAULT_SYSTEM_PROMPT + E_SYS

def get_prompt(instruction):
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

## Helper function to format the response
def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text):
    prompt = get_prompt(text)
    with torch.autocast('cuda', dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
        outputs = model.generate(**inputs,
                                 max_new_tokens=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '</s>')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        return wrapped_text


In [30]:
# Format the prompt to include the query and some context
def format_prompt(query, context):
  return '''
    ### Messages:
    {context}

    ### Question:
    {query}
    '''.format(context=context, query=query)

In [49]:
def semantic_search(query):
  data = get_data_from_chroma(query)

  context = data['context']
  score = data['score']
  metadata = data['metadata']

  prompt = format_prompt(query, context)
  generated_text = generate(prompt)
  parsed_text = parse_text(generated_text)

  return {'response': parsed_text, 'metadata': metadata, 'score': score}

In [50]:
%%time
# semantic_search("What are some models that are comparable to GPT 3?")
# semantic_search("How can I make some pancakes?")
semantic_search("What did Tollan say about semantic search?")

/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')


CPU times: user 7.92 s, sys: 79.2 ms, total: 8 s
Wall time: 9.94 s


{'response': "  Based on the chat messages provided, Tollan mentioned that semantic search is best tested with a\nlarge amount of data. They also mentioned that the search probably won't work properly with a small\namount of data. Therefore, my response would be: Tollan said that semantic search works best with a\nlarge amount of data.",
 'metadata': [{'channel': 'general',
   'date': '2023-06-19',
   'message': "Tollan: It's best if we just post random topics here to test the semantic search.",
   'time': '1687166901.338569',
   'user_id': 'U05CQ93C3FZ',
   'user_name': 'Tollan'},
  {'channel': 'general',
   'date': '2023-06-19',
   'message': "Tollan: Oh.. u r right.. the search probably won't work properly with a small amount of data.",
   'time': '1687168952.621439',
   'user_id': 'U05CQ93C3FZ',
   'user_name': 'Tollan'},
  {'channel': 'general',
   'date': '2023-06-19',
   'message': 'Tollan: Also, welcome <@U05D4M7RGQ3>',
   'time': '1687169174.199539',
   'user_id': 'U05CQ93C3FZ

## 7. Creating an API Endpoint

In [ ]:
from flask import Flask, request
from pyngrok import ngrok

port_no = 5000

app = Flask(__name__)
ngrok.set_auth_token("2S3sOVlp9tBJmDvErbUdCPoE2ri_3yrw7sVk12An4TZieirkT")
public_url =  ngrok.connect(port_no).public_url

@app.route("/", methods=['GET', 'POST'])
def semantic_search_query():

  if request.method == 'GET':
    query = request.args.get('query')
    return semantic_search(query)

  elif request.method == 'POST':
    query = request.json['query']
    return semantic_search(query)


print(f"Public url for the API... {public_url}")

app.run(port=port_no)